In [1]:
pip install verstack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 975.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 27.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 27.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 9.7 MB/s eta 0:00:00
  Created wheel for verstack: filename=verstack-3.4.0-py3-none-any.whl size=88899 sha256=f1d87cbc102bdeae0b6496

In [2]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.set_option('max_columns', None)
import warnings
warnings.filterwarnings('ignore')

import joblib

/kaggle/input/michaels-order1/Stacker_2.pkl
/kaggle/input/michaels-order1/Stacker_1.pkl
/kaggle/input/michaels-order1/stacked_valid_2
/kaggle/input/michaels-order1/stacked_valid_1
/kaggle/input/michaels-order1/stacked_train_1
/kaggle/input/michaels-order1/stacked_train_2
/kaggle/input/daconio-117/sample_submission.csv
/kaggle/input/daconio-117/train_df.csv
/kaggle/input/daconio-117/train.csv
/kaggle/input/daconio-117/test.csv
/kaggle/input/daconio-117/test_df.csv
/kaggle/input/only-stocked-df/only_stacked_train_full_feature.csv
/kaggle/input/only-stocked-df/only_stacked_train_test_feature.csv
/kaggle/input/only-stocked-df/only_stacked_test.csv


1. train null 값 처리와 feature에 대한 scaling

2. train_valid 나누기: train_set과 valid set나눈 후 저장 하던지 random_state로 통일

3. 전체 특성을 이용하여 train_set을 Stacker로 fit_transform해 줌 => Stacker_1, stacked_train_1라 명명 후 둘 다 저장

4. 테스트 특성만 이용하여 train_set을 Stacker로 fit_transform해 줌 => Stacker_2, stacked_train_2라 명명 후 둘 다 저장

5. Stacker_1을 이용하여 valid_set transform해주어 stacked_valid_1을 만듦 => stacked_valid_1 저장 + Stacker_1을 저장 후 호출하는 함수 만들기

6. Stacker_2를 이용하여 valid_set transform해주어 stacked_valid_2를 만듦 => stacked_valid_2 저장 + Stacker_2를 저장 후 호출하는 함수 만들기

7. stacked_train_1을 이용하여 ml_1모델(1차적으로 LGBM Tuner로, 최종적으로 앙상블한 최종 모델 사용할 예정)을 훈련시킨 후 
   stacked_valid_1을 이용하여 valid_score를 작성=> valid_score 저장
   
8. stacked_train_2를 이용하여 ml_2모델(7번과 동일)을 훈련시킨 후
   stacked_valid_2를 이용하여 valid_score를 작성=> valid_score 저장
   
9. Stacker_2를 이용하여 test셋을 transform해준 뒤 ml_2를 이용하여 예측한 값을 KD 전 submission으로 제출함=>submission 저장

## **1. train null 값 처리와 feature에 대한 scaling**

In [3]:
train_df = pd.read_csv('/kaggle/input/daconio-117/train_df.csv')
print(train_df.shape)
train_df.head()

(14067, 56)


,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,COMPONENT_ARBITRARY_COMPONENT1,COMPONENT_ARBITRARY_COMPONENT2,COMPONENT_ARBITRARY_COMPONENT3,COMPONENT_ARBITRARY_COMPONENT4,Y_LABEL
0,-0.915023,-0.669031,0.320692,-0.623976,-0.15342,0.144691,0.963226,-0.494248,-0.054482,1.142569,-0.103225,-0.139071,2.155068,1.482724,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,1.856659,-0.132286,-0.097601,2.878300,1.858988,0.524420,2.377223,-0.372594,1.982981,3.081712,1.417535,-0.695598,2.764586,1.001988,-0.457625,3.175785,1.236100,-0.463603,3.267063,-0.109683,-0.153248,-0.258734,-0.377342,-0.391875,-0.393032,-0.411170,-0.416807,-0.154662,-0.057476,1.012713,-0.966497,0,0,1,0,0
1,-1.079784,1.852946,3.161795,0.405744,-0.15342,-0.117202,0.116948,-0.494248,-0.054482,1.087883,-0.103225,-0.139071,-0.894855,0.913611,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,-1.762930,-0.132286,-0.097601,-0.931546,-0.260909,-0.838554,-0.788343,-0.776308,-0.434150,-0.445175,-0.378768,0.815365,-0.523272,-1.169611,-0.457625,-1.689316,-0.550961,-0.463603,-0.264231,-0.109683,-0.153248,1.051451,0.721457,0.970117,0.818675,1.550216,1.762718,-0.154662,-0.057476,-1.943832,0.118657,0,1,0,0,0
2,-0.081302,0.339760,-1.168053,-0.623976,-0.15342,3.170023,-1.142211,0.748363,-0.054482,-0.911187,-0.103225,-0.139071,-0.093794,-0.577088,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,-1.438062,-0.132286,-0.097601,-0.931546,-0.260909,-1.307430,-0.788343,-0.776308,0.023809,-0.445175,-1.191871,-0.695598,-0.580505,-1.146343,-0.457625,-1.689316,0.576549,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,-0.377342,0.781281,1.904013,2.990037,2.762455,-0.154662,-0.057476,-0.767722,-0.332706,0,1,0,0,1
3,1.839183,-0.921229,-1.168053,-0.623976,-0.15342,0.882927,-0.763166,-0.494248,-0.054482,0.400597,-0.103225,-0.139071,-0.894855,-0.832430,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,-0.148229,-0.132286,-0.097601,-0.931546,-0.260909,-0.218732,-0.013886,-0.776308,-1.217035,-0.445175,0.294274,0.257713,-0.082769,1.045306,1.984632,-1.132762,0.576549,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,-0.377342,-0.391875,-0.393032,-0.411170,-0.416807,-0.154662,-0.057476,0.670030,-1.094383,0,0,1,0,0
4,0.765327,0.339760,-0.392700,-0.623976,-0.15342,-0.486320,1.247203,-0.494248,-0.054482,-0.874730,-0.103225,-0.139071,-0.894855,-1.268940,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,0.258203,-0.132286,-0.097601,-0.931546,-0.260909,-1.307430,-0.788343,-0.776308,0.023809,-0.445175,-1.279047,-0.695598,0.774321,0.691040,-0.457625,-0.807199,-0.550961,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,-0.377342,-0.391875,-0.393032,-0.411170,-0.416807,-0.154662,-0.057476,0.666467,-0.866820,0,0,1,0,0


**null값 drop 및 scaling 모두 마친 train set**

## **2. train_valid 나누기: train_set과 valid set나눈 후 저장 하던지 random_state로 통일**

In [4]:
from sklearn.model_selection import train_test_split

feature = train_df.drop(['Y_LABEL'], axis=1)
target = train_df['Y_LABEL']

X_train, X_valid, y_train, y_valid = train_test_split(feature, target, test_size=0.2, random_state=42)

**random_state를 비롯한 seed들은 앞으로 42로 고정 이러면 굳이 valid set 저장할 필요 없을 듯**

## **3. 전체 특성을 이용하여 train_set을 Stacker로 fit_transform해 줌 => Stacker_1, stacked_train_1라 명명 후 둘 다 저장**

In [5]:
#index reset
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_train.head(3)

,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,COMPONENT_ARBITRARY_COMPONENT1,COMPONENT_ARBITRARY_COMPONENT2,COMPONENT_ARBITRARY_COMPONENT3,COMPONENT_ARBITRARY_COMPONENT4
0,-0.023281,-1.173427,1.711478,1.034058,-0.15342,-0.117202,-0.641141,-0.494248,-0.054482,0.592335,-0.103225,-0.139071,-0.894855,1.969028,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,-0.556427,-0.132286,-0.097601,-0.931546,-0.260909,-1.307430,-0.788343,-0.372594,-1.217035,-0.445175,-0.759178,0.257713,-0.800756,-1.148595,-0.457625,-1.689316,-0.550961,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,1.702452,1.467534,2.480778,2.284467,2.529955,-0.154662,-0.057476,-1.927948,0.118657,0,1,0,0
1,-1.105403,-0.416834,-0.731397,-0.623976,-0.15342,0.347832,1.110267,0.748363,-0.054482,1.230336,-0.103225,-0.139071,0.707267,1.592205,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,1.981607,-0.132286,-0.097601,0.324543,-0.260909,0.869967,3.184068,0.800960,0.023809,1.546017,0.590266,0.257713,2.409786,-0.435880,1.083268,0.631473,2.363609,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,-0.377342,-0.391875,-0.393032,-0.411170,-0.416807,-0.154662,-0.057476,-1.570778,1.369311,0,0,1,0
2,0.153575,-0.164636,0.659389,-0.623976,-0.15342,3.425744,1.049182,-0.494248,-0.054482,-0.882156,-0.103225,-0.139071,-0.093794,-0.255395,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,0.749926,-0.132286,-0.097601,0.653461,-0.260909,-0.369677,0.439143,-0.776308,0.600769,-0.445175,1.728853,-0.695598,0.886489,2.487035,-0.457625,0.631473,-0.550961,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,-0.377342,-0.391875,-0.393032,-0.411170,-0.416807,-0.154662,-0.057476,0.724583,-1.054889,0,0,1,0


In [6]:
from verstack import Stacker

Stacker_1 = Stacker(objective = 'binary', auto = True)
stacked_train_1 = Stacker_1.fit_transform(X_train, y_train)

2022-12-10 06:34:56.587433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 06:34:56.760437: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-10 06:34:56.760491: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-10 06:34:58.000713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 


 * Initiating Stacker.fit_transform

   - Training/predicting with layer_1 models
     . Optimising model hyperparameters
     .. fold 2 trained/predicted
     .. fold 4 trained/predicted
     . Optimising model hyperparameters
     .. fold 2 trained/predicted
     .. fold 4 trained/predicted
     . Optimising model hyperparameters
     .. fold 2 trained/predicted
     .. fold 4 trained/predicted
     . Optimising model hyperparameters
     .. Model not in optimisation list <verstack.stacking.kerasModel.kerasModel object at 0x7f972ca39f10>


2022-12-10 06:39:34.725891: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-10 06:39:34.725995: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-10 06:39:34.726036: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c806563d0f82): /proc/driver/nvidia/version does not exist
2022-12-10 06:39:34.726574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other 

Epoch 1/200
211/211 [==============================] - 2s 3ms/step - loss: 0.2926 - val_loss: 0.1651
Epoch 2/200
211/211 [==============================] - 0s 2ms/step - loss: 0.1765 - val_loss: 0.1590
Epoch 3/200
211/211 [==============================] - 0s 2ms/step - loss: 0.1691 - val_loss: 0.1572
Epoch 4/200
211/211 [==============================] - 1s 2ms/step - loss: 0.1625 - val_loss: 0.1576
Epoch 5/200
211/211 [==============================] - 1s 2ms/step - loss: 0.1575 - val_loss: 0.1570
Epoch 6/200
211/211 [==============================] - 1s 2ms/step - loss: 0.1533 - val_loss: 0.1586
Epoch 7/200
211/211 [==============================] - 1s 2ms/step - loss: 0.1485 - val_loss: 0.1595
Epoch 8/200
211/211 [==============================] - 1s 2ms/step - loss: 0.1424 - val_loss: 0.1608
Epoch 9/200
211/211 [==============================] - 1s 3ms/step - loss: 0.1390 - val_loss: 0.1694
Epoch 10/200
211/211 [==============================] - 1s 3ms/step - loss: 0.1345 - val_lo

In [7]:
print(stacked_train_1.shape)
stacked_train_1.head(3)

(11253, 167)


,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,COMPONENT_ARBITRARY_COMPONENT1,COMPONENT_ARBITRARY_COMPONENT2,COMPONENT_ARBITRARY_COMPONENT3,COMPONENT_ARBITRARY_COMPONENT4,layer_1_0,layer_1_1,layer_1_2,layer_1_3,layer_1_4,layer_1_5,layer_1_6,layer_1_7,layer_1_8,layer_1_9,layer_1_10,layer_1_11,layer_1_12,layer_1_13,diff_layer_1_0_layer_1_1,diff_layer_1_0_layer_1_2,diff_layer_1_0_layer_1_3,diff_layer_1_0_layer_1_4,diff_layer_1_0_layer_1_5,diff_layer_1_0_layer_1_6,diff_layer_1_0_layer_1_7,diff_layer_1_0_layer_1_8,diff_layer_1_0_layer_1_9,diff_layer_1_0_layer_1_10,diff_layer_1_0_layer_1_11,diff_layer_1_0_layer_1_12,diff_layer_1_0_layer_1_13,diff_layer_1_1_layer_1_2,diff_layer_1_1_layer_1_3,diff_layer_1_1_layer_1_4,diff_layer_1_1_layer_1_5,diff_layer_1_1_layer_1_6,diff_layer_1_1_layer_1_7,diff_layer_1_1_layer_1_8,diff_layer_1_1_layer_1_9,diff_layer_1_1_layer_1_10,diff_layer_1_1_layer_1_11,diff_layer_1_1_layer_1_12,diff_layer_1_1_layer_1_13,diff_layer_1_2_layer_1_3,diff_layer_1_2_layer_1_4,diff_layer_1_2_layer_1_5,diff_layer_1_2_layer_1_6,diff_layer_1_2_layer_1_7,diff_layer_1_2_layer_1_8,diff_layer_1_2_layer_1_9,diff_layer_1_2_layer_1_10,diff_layer_1_2_layer_1_11,diff_layer_1_2_layer_1_12,diff_layer_1_2_layer_1_13,diff_layer_1_3_layer_1_4,diff_layer_1_3_layer_1_5,diff_layer_1_3_layer_1_6,diff_layer_1_3_layer_1_7,diff_layer_1_3_layer_1_8,diff_layer_1_3_layer_1_9,diff_layer_1_3_layer_1_10,diff_layer_1_3_layer_1_11,diff_layer_1_3_layer_1_12,diff_layer_1_3_layer_1_13,diff_layer_1_4_layer_1_5,diff_layer_1_4_layer_1_6,diff_layer_1_4_layer_1_7,diff_layer_1_4_layer_1_8,diff_layer_1_4_layer_1_9,diff_layer_1_4_layer_1_10,diff_layer_1_4_layer_1_11,diff_layer_1_4_layer_1_12,diff_layer_1_4_layer_1_13,diff_layer_1_5_layer_1_6,diff_layer_1_5_layer_1_7,diff_layer_1_5_layer_1_8,diff_layer_1_5_layer_1_9,diff_layer_1_5_layer_1_10,diff_layer_1_5_layer_1_11,diff_layer_1_5_layer_1_12,diff_layer_1_5_layer_1_13,diff_layer_1_6_layer_1_7,diff_layer_1_6_layer_1_8,diff_layer_1_6_layer_1_9,diff_layer_1_6_layer_1_10,diff_layer_1_6_layer_1_11,diff_layer_1_6_layer_1_12,diff_layer_1_6_layer_1_13,diff_layer_1_7_layer_1_8,diff_layer_1_7_layer_1_9,diff_layer_1_7_layer_1_10,diff_layer_1_7_layer_1_11,diff_layer_1_7_layer_1_12,diff_layer_1_7_layer_1_13,diff_layer_1_8_layer_1_9,diff_layer_1_8_layer_1_10,diff_layer_1_8_layer_1_11,diff_layer_1_8_layer_1_12,diff_layer_1_8_layer_1_13,diff_layer_1_9_layer_1_10,diff_layer_1_9_layer_1_11,diff_layer_1_9_layer_1_12,diff_layer_1_9_layer_1_13,diff_layer_1_10_layer_1_11,diff_layer_1_10_layer_1_12,diff_layer_1_10_layer_1_13,diff_layer_1_11_layer_1_12,diff_layer_1_11_layer_1_13,diff_layer_1_12_layer_1_13,layer_1_std,layer_1_mean,layer_2_0,layer_2_1,diff_layer_2_0_layer_2_1,layer_2_std,layer_2_mean
0,-0.023281,-1.173427,1.711478,1.034058,-0.15342,-0.117202,-0.641141,-0.494248,-0.054482,0.592335,-0.103225,-0.139071,-0.894855,1.969028,0.142333,0.015145,-0.393591,-0.019127,-0.010975,-0.12846,-0.556427,-0.132286,-0.097601,-0.931546,-0.260909,-1.307430,-0.788343,-0.372594,-1.217035,-0.445175,-0.759178,0.257713,-0.800756,-1.148595,-0.457625,-1.689316,-0.550961,-0.463603,-0.264231,-0.109683,-0.153248,-0.258734,1.702452,1.467534,2.480778,2.284467,2.529955,-0.154662,-0.057476,-1.927948,0.118657,0,1,0,0,0.050038,0.101665,0.013456,1.142380e-05,1.0,4.921171e-05,0.000000,0.046065,0.034407,0.058801,0.08666,0.0,0.0,0.015896,-0.051627,0.036582,0.050026,-0.949962,0.049989,0.050038,0.003973,0.015631,-0.008764,-0.036622,0.050038,0.050038,0.034142,0.088208,0.101653,-0.898335,0.101615,0.101665,0.055600,0.067258,0.042863,0.015005,0.101665,0.101665,0.085769,0.013445,-0.986544,0.013407,0.013456,-0.032609,-0.020951,-0.045345,-0.073204,0.013456,0.013456,-0.002440,-0.999989,-3.778791e-05,0.000011,-0.046053,-0.034396,-0.058790,-8.664836e-02,1.142380e-05,1.142380e-05,-0.015885,0.

In [8]:
stacked_train_1.to_csv('stacked_train_1', index=False)

In [16]:
joblib.dump(Stacker_1, 'Stacker_1.h5')
Stacker_1

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......normalization
.........vars
............0
............1
............2
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-12-10 07:36:55         2825
variables.h5                                   2022-12-10 07:36:55        98832
metadata.json                                  2022-12-10 07:36:55  

Stacker(objective: binary            
        auto: True            
        num_auto_layers: 2            
        metafeats: True            
        epochs : 200            
        gridsearch_iterations: 10            
        stacking_feats_depth: 1            
        include_X: False            
        verbose : True)

In [10]:
Stacker_1_trained_models = Stacker_1.trained_models

In [11]:
joblib.dump(Stacker_1_trained_models, 'Stacker_1_trained_models.pkl')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......normalization
.........vars
............0
............1
............2
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-12-10 07:03:12         2825
variables.h5                                   2022-12-10 07:03:12        98832
metadata.json                                  2022-12-10 07:03:12  

['Stacker_1_trained_models.pkl']

In [15]:
Stacker_1

Stacker(objective: binary            
        auto: True            
        num_auto_layers: 2            
        metafeats: True            
        epochs : 200            
        gridsearch_iterations: 10            
        stacking_feats_depth: 1            
        include_X: False            
        verbose : True)

## **4. 테스트 특성만 이용하여 train_set을 Stacker로 fit_transform해 줌 => Stacker_2, stacked_train_2라 명명 후 둘 다 저장**

test_df = pd.read_csv('/kaggle/input/daconio-117/test_df.csv')
test_col = test_df.columns
X_train_test_col = X_train[test_col]
X_train_test_col.head(3)

Stacker_2 = Stacker(objective = 'binary', auto = True)
stacked_train_2 = Stacker_2.fit_transform(X_train_test_col, y_train)

print(stacked_train_2.shape)
stacked_train_2.head(3)

stacked_train_2.to_csv('stacked_train_2', index=False)

joblib.dump(Stacker_2, 'Stacker_2.pkl')
Stacker_2

## **5. Stacker_1을 이용하여 valid_set transform해주어 stacked_valid_1을 만듦 => stacked_valid_1 저장 후 호출하는 함수 만들기**

X_valid.reset_index(inplace=True, drop=True)
y_valid.reset_index(drop=True, inplace=True)
X_valid.head(3)

stacked_valid_1 = Stacker_1.transform(X_valid)

print(stacked_valid_1.shape)
stacked_valid_1.head()

stacked_valid_1.to_csv('stacked_valid_1', index=False)

## **6. Stacker_2를 이용하여 valid_set transform해주어 stacked_valid_2를 만듦 => stacked_valid_2 저장 후 호출하는 함수 만들기**

X_valid_test_col = X_valid[test_col]
X_valid_test_col

stacked_valid_2 = Stacker_2.transform(X_valid_test_col)

print(stacked_valid_2.shape)
stacked_valid_2.head(3)

stacked_valid_2.to_csv('stacked_valid_2', index=False)

## **7. stacked_train_1을 이용하여 ml_1모델(1차적으로 LGBM Tuner로, 최종적으로 앙상블한 최종 모델 사용할 예정)을 훈련시킨 후 stacked_valid_1을 이용하여 valid_score를 작성=> valid_score 저장**

'''/kaggle/input/michaels-order1/Stacker_2.pkl
/kaggle/input/michaels-order1/Stacker_1.pkl
/kaggle/input/michaels-order1/stacked_valid_2
/kaggle/input/michaels-order1/stacked_valid_1
/kaggle/input/michaels-order1/stacked_train_1
/kaggle/input/michaels-order1/stacked_train_2'''

Stacker_2 = joblib.load('/kaggle/input/michaels-order1/Stacker_2.pkl')
Stacker_1 = joblib.load('/kaggle/input/michaels-order1/Stacker_1.pkl')

stacked_valid_2 = pd.read_csv('/kaggle/input/michaels-order1/stacked_valid_2')
stacked_valid_1 = pd.read_csv('/kaggle/input/michaels-order1/stacked_valid_1')
stacked_train_1 = pd.read_csv('/kaggle/input/michaels-order1/stacked_train_1')
stacked_train_2 = pd.read_csv('/kaggle/input/michaels-order1/stacked_train_2')